## Idea

### General

We already have standby servers that are *almost* (eventually) consistent with the primary server.

We can use them to create a distributed DB system to enhance performance.

### Types Of Distributed DB Architectures

- Single Write / Read, Multiple Read (Most Common due to low rate of conflicts)
- Multiple Write / Read (can work with more Read Only as well)
- Partitioned Data By Server - every server saves a partition of the data, for example the relevant country data so the latency is lower due to less network distance

## Single Write And Multiple Read

### Conflicts

#### Example

<img src="./helpers/Replication - Conflicts.png" alt="drawing"/>

### Delays

Postgres should decide either to grace the running `SELECT` on standby server or cancel the query and apply the conflicting WAL from primary.

For short queries it probably worth it to wait to smoothen the experience for the user, but on long running queries the standby can fall behind on WAL replay and become not up to date.

To manipulate what is considered a "short" query: 
- `max_standby_archive_delay`
- `max_standby_streaming_delay`

In a standby server that exists primarily for high availability, it's best to set the delay parameters relatively short, so that the server cannot fall far behind the primary due to delays caused by standby queries. However, if the standby server is meant for executing long-running queries, then a high or even infinite delay value may be preferable. Keep in mind however that a long-running query could cause other sessions on the standby server to not see recent changes on the primary, if it delays application of WAL records.

#### Main Types

- Changes due to MVCC maintenance like removing dead rows and changing the visibility map
- Changes of the DDL
- Locks

#### Monitor

View `pg_stat_database_conflicts`